In [1]:
import numpy as np
import pandas as pd
import os
import re
import random
from typing import List, Dict
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm

from als import ALS

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
def get_data(data_dir: str):
  partition_files = []

  pattern = re.compile(r'part_(\d+)_(\d+)\.parquet$')

  for filename in os.listdir(data_dir):
    match = pattern.match(filename)
    if match:
        file_path = os.path.join(data_dir, filename)
        part_num = int(match.group(1))
        group_num = int(match.group(2))

        partition_files.append({
            'path': file_path,
            'part': part_num,
            'group': group_num
        })

  # sorted partitions in order
  partition_files.sort(key=lambda x: (x['part'], x['group']))
  return partition_files

In [4]:
def map_id(partition_files: List[Dict], sample_size=None):
  """
  Map user and movie ids to denser indicies -> for matrix factorization
  """

  user_ids = set()
  movie_ids = set()

  if sample_size and sample_size < len(partition_files):
    partition_sample = random.sample(partition_files, sample_size)
  else:
    partition_sample = partition_files

  for partition in tqdm(partition_sample, desc="Mapping IDs"):
    df = pd.read_parquet(partition['path'], columns=['user_id', 'movie_id'])

    user_ids.update(df['user_id'].unique())
    movie_ids.update(df['movie_id'].unique())

  user_id_map = {user_id: idx for idx, user_id in enumerate(sorted(user_ids))}
  movie_id_map = {movie_id: idx for idx, movie_id in enumerate(sorted(movie_ids))}

  print(f"Map successful for {len(user_id_map)} users, {len(movie_id_map)} movies")

  return user_id_map, movie_id_map

In [5]:
def evaluate_model(model, validation_partitions):
    """Evaluate ALS model on validation partitions"""
    all_errors = []

    for partition in tqdm(validation_partitions, desc="Evaluating validation partitions"):
        # Load validation data
        df = pd.read_parquet(partition['path'], columns=['user_id', 'movie_id', 'rating'])

        # Get predictions
        predictions = model.batch_predict(df)

        # Calculate squared errors
        errors = (predictions - df['rating'].values) ** 2
        all_errors.extend(errors)

    # Calculate RMSE
    rmse = np.sqrt(np.mean(all_errors))
    print(f"Validation RMSE: {rmse:.4f}")

    return rmse

In [6]:
def get_model_save_path(dir):
  i = 1
  while True:
    save_path = os.path.join(dir, f'model_{i}.pkl')
    if not os.path.exists(save_path):
      return save_path
    i += 1

In [7]:
data_dir = '/gdrive/MyDrive/Netflix_Prize/split/train'
output_dir = '/gdrive/MyDrive/Netflix_Prize/results'
val_dir = '/gdrive/MyDrive/Netflix_Prize/split/validation'
checkpoint_dir = '/gdrive/MyDrive/Netflix_Prize/checkpoints'

In [8]:
# retreive training data info
train_partition_files = get_data(data_dir)
print(f"Number of training partitions: {len(train_partition_files)}")
val_partition_files = get_data(val_dir)
print(f"Number of validation partitions: {len(val_partition_files)}")

seed = 42
random.seed(seed)

# testing
sample_train_partitions = random.sample(train_partition_files, 7)

sample_val_partitions = []
for partition in sample_train_partitions:
  val_partition = partition.copy()
  val_partition['path'] = partition['path'].replace('train', 'validation')
  sample_val_partitions.append(val_partition)


print(f"Train EX: {sample_train_partitions[0]}")
print(f"Val EX: {sample_val_partitions[0]}")

Number of training partitions: 34
Number of validation partitions: 34
Train EX: {'path': '/gdrive/MyDrive/Netflix_Prize/split/train/part_1_7.parquet', 'part': 1, 'group': 7}
Val EX: {'path': '/gdrive/MyDrive/Netflix_Prize/split/validation/part_1_7.parquet', 'part': 1, 'group': 7}


In [9]:
# build user_map and movie_map
user_map, movie_map = map_id(sample_train_partitions)

Mapping IDs: 100%|██████████| 7/7 [00:06<00:00,  1.07it/s]


Map successful for 431670 users, 3720 movies


In [10]:
# 7 each as a test + for tuning
train_partitions = sample_train_partitions
val_partitions = sample_val_partitions

In [13]:
# Tuning grid
param_grid = {
    'num_factors': [10, 20, 30],
    'lambda_reg': [0.01, 0.1, 1],
    'num_iters': [15, 20]
}

In [12]:
# init model - small test values

model = ALS(
    num_factors=50,
    lambda_reg=0.1,
    num_iters=15,
    val_interval=5
)

model.fit(
    partitions=train_partitions,
    user_map=user_map,
    movie_map=movie_map,
    checkpoint_dir=checkpoint_dir
)

model.save_model(get_model_save_path(output_dir))

val_rmse = evaluate_model(model, val_partitions)

Building sparse matrices for 431670 users, and 3720 movies


ALS Optimization:  27%|██▋       | 4/15 [11:55<32:56, 179.67s/it]


Saved checkpoint to /gdrive/MyDrive/Netflix_Prize/checkpoints/checkpoint_5.npz, in 11.142310380935669 seconds


ALS Optimization:  33%|███▎      | 5/15 [15:04<30:33, 183.34s/it]


Iteration 5 -- RMSE: 0.5020


...

ALS Optimization: 100%|██████████| 15/15 [45:06<00:00, 180.46s/it]


Iteration 15 -- RMSE: 0.4666



Trained Model saved at /gdrive/MyDrive/Netflix_Prize/results/model_1.pkl, in 12.369998216629028 seconds


Evaluating validation partitions: 100%|██████████| 7/7 [00:11<00:00,  1.65s/it]


Validation RMSE: 1.6540
